# 3D example sphere

The entire code with 2D and 3D plot

# Geometry

Similarly to how it was done in example_capa_parallel, we start by defining the geometry of the system, i.e.
by creating a ContinuousGeometry.

We notice here that space is not an obligatory parameter, if one gives as parameter both voltage and all charge regions space is not needed. 

In [ ]:
from poisson.continuous import shapes
from poisson import ContinuousGeometry

epislon = 1e-14 # Is this still nescessary -> Ask pacome to se if he fixed this
r1 = 3.0 + epislon
r2 = 8.0 + epislon
r3 = 10.0 + epislon

center = [0.0, 0.0, 0.0]
cercle1 = shapes.Ellipsoid((r1, r1, r1), center)
cercle2 = (shapes.Ellipsoid((r2, r2, r2), center)
           - shapes.Ellipsoid((r1, r1, r1), center))
cercle3 = (shapes.Ellipsoid((r3, r3, r3), center)
           - shapes.Ellipsoid((r2, r2, r2), center))

# Construct the poisson problem
geometry = ContinuousGeometry(voltage={'elect1':cercle1, 'elect2':cercle3}, 
                              charge={'insulator':cercle2})


We then create a GridBuilder instance to generate the points in the mesh. 
Here we are going to make a spherical mesh instead of a rectangular mesh. 

In order to do the latter GridBuilder.add_mesh_spherical must be used. If a circular mesh is to be used (in a 2D case) use GridBuilder.add_mesh_circular()

Moreover, the step will not be the same in all directions, as we can define the step in each direction separetly. 

Additionally, since we are particularly concerned with what happens at the interface betwen two regions, e.g. the electrode and the insulator, we will set the step at the interface to be considerably smaller than an the interior of each zone. 

In [ ]:
from poisson import GridBuilder
import numpy as np

# Create the shapes instances around the two interfaces elc1 - insulator and elect2 - insulator
r1_2a = 2.90 + epislon
r1_2b = 3.1 + epislon
r2_3a = 7.90 + epislon
r2_3b = 8.1 + epislon
cercle_1_2 = (shapes.Ellipsoid((r1_2b, r1_2b, r1_2b), center)
              - shapes.Ellipsoid((r1_2a, r1_2a, r1_2a), center))
cercle_2_3 = (shapes.Ellipsoid((r2_3b, r2_3b, r2_3b), center)
              - shapes.Ellipsoid((r2_3a, r2_3a, r2_3a),center))

# Defined the step along each direction
sphericalbox = [-10, 10, 0, 2 * np.pi, 0, np.pi] # 1 -> x, 2 -> y, 3 -> z
step = [0.1, np.pi / 10, np.pi / 10]
step_2 = [0.01, np.pi/ 10, np.pi/ 10]

# Build the grid 
grid = GridBuilder(build_mesh=False)
grid.add_mesh_spherical(sphericalbox, step, cercle1, 0)
grid.add_mesh_spherical(sphericalbox, step, cercle2, 1)
grid.add_mesh_spherical(sphericalbox, step, cercle3, 2)
grid.add_mesh_spherical(sphericalbox, step_2, cercle_1_2, 3)
grid.add_mesh_spherical(sphericalbox, step_2, cercle_2_3, 4)

Visualize the continuous geometry

Here one can use the function in each class or use the plot module poisson.plot

In [ ]:
from poisson import plot as p_plt
import matplotlib.pyplot as plt

# Show the grid
p_plt.points_3D_mavi(cls_inst=grid, scale_factor = 0.05)
plt.show()

# Show the continuous problem

# 3D
geometry.plot(points=grid.points, plot_type='3D', scale_factor = 0.05)
plt.show()
# or
p_plt.points_3D_mavi(cls_inst=geometry, points=grid.points,
                     scale_factor = 0.05)

# 2D
bbox = [-10, 10, -10, 10]
plot_geometry = geometry.plot(direction=2, bbox=bbox, plot_type='2D')
plot_geometry(variable=0)
#or
plot_geometry = p_plt.plot_continuous_geometry(geometry_inst=geometry,
                                               bbox=bbox)
plot_geometry(variable=0)
plt.show()

# Discrete Problem and solving it

In [ ]:
from poisson import DiscretePoisson, LinearProblem

# As it was exaplained in example_capa_parallel

sys_instance = DiscretePoisson(
        geometry, grid=grid,
        selection={'Neuman-Dirichlet':[['voltage', '*']]})

linear_prob_inst = LinearProblem(sys_instance, is_charge_density=True,
                                 voltage_val=[(cercle3, 4.0),
                                              (cercle1, 0.0)])

# Plotting results

In [ ]:
import matplotlib.pyplot as plt

#### Plotting
# Plot 3D 
linear_prob_inst.plot_3d(plot_='both', titles=('voltage', 'charge'), scale_factor=(0.1, 0.01))

# Plot 2D cut
plot_volt, plot_charge = linear_prob_inst.plot_cut_2d(direction=2,
                                                      npoints=(1000,1000))
plot_volt(variable=0, colorbar_label='Voltage (V)')
plot_charge(variable=0, colorbar_label='Charge')
plt.show()

# Plot 1D cut
plot_voltage, plot_charge = linear_prob_inst.plot_cut_1d(
        directions=(1, 2), bbox='default', npoints=2000)

fig2 = plt.figure()
ax_voltage = fig2.add_subplot(111)

t, data_volt = plot_voltage(
        (0, 0), ax = ax_voltage, marker='.', color='k',
        label='Voltage simulation', linestyle='None')

ax_voltage.set_xlabel('{0}(nm)'.format('y'))
ax_voltage.set_ylabel('Voltage(V)')
ax_voltage.legend(loc='upper center')

ax_charge = ax_voltage.twinx()

tt, data_charge = plot_charge(
        (0, 0), ax = ax_charge, marker='.', color='b',
        label='Charge simulation', linestyle='None')

ax_charge.set_xlabel('{0}(nm)'.format('y'))
ax_charge.set_ylabel(r'Charge density $(\#.nm^{{{-2}}})$')
ax_charge.legend(loc='lower center')

plt.show()